In [29]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', None)


![CRISP_DM](Crisp_DM_Tasks.png)

# Group-by apply on large (relational) data set

In [30]:
pd_JH_data=pd.read_csv('C:/ProgramData/Anaconda3/eps_covid19/data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
169,2020-01-22,no,"Korea, South",1
170,2020-01-22,no,Kosovo,0
171,2020-01-22,no,Kuwait,0
172,2020-01-22,no,Kyrgyzstan,0


In [31]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-07-30')]

In [32]:
test_data.head()

,date,state,country,confirmed
50950,2020-07-31,no,Germany,210399
51058,2020-07-31,no,US,4562107
51216,2020-08-01,no,Germany,211005
51324,2020-08-01,no,US,4620592
51482,2020-08-02,no,Germany,211220


In [33]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-09-13,no,261737
US,2020-09-13,no,6519573


In [34]:
# %load C:/ProgramData/Anaconda3/eps_covid19/src/features/build_features.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate
        Parameters:
        ----------
        in_array : pandas.series
        Returns:
        ----------
        Doubling rate: double
    '''

    #print(in_array)
    #print(type)
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    #y = np.array(in_array['confirmed'])
    #X = np.arange(len(y)).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))


the test slope is: [2.]


In [35]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-09-13     261737
      US      2020-09-13    6519573

In [36]:
# this command will only work when adapting the get_doubling_time_via_regression function

#test_data.groupby(['state','country']).apply(get_doubling_time_via_regression)

In [37]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [38]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  50950           NaN
                51216           NaN
                51482    513.701989
                51748    382.360458
                52014    263.747512
                52280    212.804529
                52546    193.571536
                52812    206.544407
                53078    231.809013
                53344    397.061661
                53610    271.110696
                53876    194.001184
                54142    186.844463
                54408    161.177186
                54674    151.644156
                54940    202.555505
                55206    369.126096
                55472    203.795961
                55738    145.588821
                56004    151.813262
                56270    144.833544
                56536    139.237035
                56802    181.181264
                57068    319.173606
                57334    207.718414
                57600    152.843207
                57866    164.523315
      

In [39]:
#test_data

In [40]:
#pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

In [41]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [42]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'doubling_rate',
                            'level_2':'index'})
pd_DR_result.head()

,state,country,index,doubling_rate
0,Alberta,Canada,0,NaN
1,Alberta,Canada,266,NaN
2,Alberta,Canada,532,NaN
3,Alberta,Canada,798,NaN
4,Alberta,Canada,1064,NaN


In [43]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,169,2020-01-22,no,"Korea, South",1
2,170,2020-01-22,no,Kosovo,0
3,171,2020-01-22,no,Kuwait,0
4,172,2020-01-22,no,Kyrgyzstan,0


In [44]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index', 'doubling_rate']],on=['index'],how='left')

In [45]:
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate
0,0,2020-01-22,Alberta,Canada,0,NaN
1,169,2020-01-22,no,"Korea, South",1,NaN
2,170,2020-01-22,no,Kosovo,0,NaN
3,171,2020-01-22,no,Kuwait,0,NaN
4,172,2020-01-22,no,Kyrgyzstan,0,NaN


In [46]:
#pd_result_larg[pd_result_larg['country']=='Germany']

# Filtering the data with groupby apply

In [47]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [48]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [49]:
#pd_filtered_result

In [50]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,169,2020-01-22,no,"Korea, South",1,NaN,-4.8
2,170,2020-01-22,no,Kosovo,0,NaN,0.0
3,171,2020-01-22,no,Kuwait,0,NaN,0.0
4,172,2020-01-22,no,Kyrgyzstan,0,NaN,10.8


# Filtered doubling rate

In [51]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
62771,no,Zimbabwe,61540,-1.489963
62772,no,Zimbabwe,61776,178.143639
62773,no,Zimbabwe,62072,199.421609
62774,no,Zimbabwe,62308,173.654778
62775,no,Zimbabwe,62775,6.874101


In [52]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,confirmed_filtered_DR
62771,62604,2020-09-13,no,Barbados,181,360.666667,10.0,-1.671083
62772,62605,2020-09-13,no,Belarus,74173,380.346187,4997.2,0.866559
62773,62606,2020-09-13,no,Belgium,93455,96.444213,3069.8,1.467330
62774,62592,2020-09-13,no,Albania,11353,67.387550,6803.4,0.689126
62775,62775,2020-09-13,no,Zimbabwe,7526,319.333333,7528.8,6.874101


In [53]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)

In [54]:
#pd_result_larg[pd_result_larg['country']=='Germany'].tail()
pd_result_larg.head()

,index,date,state,country,confirmed,doubling_rate,confirmed_filtered,confirmed_filtered_DR
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0,NaN
1,169,2020-01-22,no,"Korea, South",1,NaN,-4.8,NaN
2,170,2020-01-22,no,Kosovo,0,NaN,0.0,NaN
3,171,2020-01-22,no,Kuwait,0,NaN,0.0,NaN
4,172,2020-01-22,no,Kyrgyzstan,0,NaN,10.8,NaN


In [55]:
pd_result_larg.to_csv('C:/ProgramData/Anaconda3/eps_covid19/data/processed/COVID_final_set.csv',sep=';',index=False)